In [0]:
-- create new managed volume landing under dev >  bronze
create volume dev.bronze.landing
comment 'This is a managed landing volume'
;

In [0]:

%python
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/input")

In [0]:
%python
dbutils.fs.cp("databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/input")

In [0]:
%python
dbutils.fs.cp("databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv", "/Volumes/dev/bronze/landing/input")

In [0]:
--create placeholder table dev.bronze.invoice_cp
create table dev.bronze.invoice_cp;

In [0]:
-- use copy into to load data into place holder table
copy into dev.bronze.invoice_cp
from "/Volumes/dev/bronze/landing/input"
fileformat = csv
pattern = '*.csv'
format_options (
  'mergeSchema' = 'true',
  'header' = 'true'
)
Copy_options(
  'mergeSchema' = 'true'
);

In [0]:
-- view data from table
select * from dev.bronze.invoice_cp

In [0]:
describe extended dev.bronze.invoice_cp;

In [0]:
-- create new table with only 3 columns invoice_cp_alt and custom columns _inser_date
create table dev.bronze.invoice_cp_alt (
  InvoiceNo string,
  StockCode string,
  Quantity double,
  _insert_date timestamp
)
;

In [0]:
-- load data using copy into in the new table invoice_cp_alt
copy into dev.bronze.invoice_cp_alt
from (
select InvoiceNo, StockCode, cast(Quantity as double) as Quantity, current_timestamp() _insert_date
from
"/Volumes/dev/bronze/landing/input"
)
fileformat = csv
pattern = '*.csv'
format_options (
  'mergeSchema' = 'true',
  'header' = 'true'
)
Copy_options(
  'mergeSchema' = 'true'
);

In [0]:
select * from dev.bronze.invoice_cp_alt